# DS7331 Project 3
#### Group 2: Hollie Gardner, Cleveland Johnson, Shelby Provost
[Dataset Source](https://archive-beta.ics.uci.edu/ml/datasets/census+income)<br/>
[Github Repo](https://github.com/ShelbyP27/DS7331-Project)

In [14]:
#import libraries
import pandas as pd
import numpy as np
import os
import sklearn as sk

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# data preprocessing 
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.pipeline import Pipeline

#prediction models
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import metrics as mt
from sklearn.neighbors import KNeighborsClassifier

# cluster analysis

# arm
# https://mhahsler.github.io/arules/docs/python/arules_python.html
from rpy2.robjects import pandas2ri
pandas2ri.activate()

import rpy2.robjects as ro
from rpy2.robjects.packages import importr

arules = importr("arules")
arulesViz = importr("arulesViz")

import time
%matplotlib inline
from rpy2.robjects.packages import importr
from rpy2 import robjects as robj

%load_ext rmagic
%load_ext rpy2.ipython 

#cleveland's instructions

# these packages will need to be installed
# open R and run 
#     install.package(arules)
#     install.package(arulesViz)
#lib='C:/Users/johnc45/R/R-Library'
#arules = importr('arules', lib_loc=lib) # same as importing in R with the "library" command
#arules_viz = importr('arulesViz', lib_loc=lib) # visualize the different rules



The rmagic extension is already loaded. To reload it, use:
  %reload_ext rmagic
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [9]:
# some helper functions from michael hahsler's github -- https://mhahsler.github.io/arules/docs/python/arules_python.html
def arules_as_matrix(x, what = "items"):
    return ro.r('function(x) as(' + what + '(x), "matrix")')(x)

def arules_as_dict(x, what = "items"):
    l = ro.r('function(x) as(' + what + '(x), "list")')(x)
    l.names = [*range(0, len(l))]
    return dict(zip(l.names, map(list,list(l))))

def arules_quality(x):
    return x.slots["quality"]

## Data Preparation

### Loading and Prepping Data 


In [10]:
# Importing the census dataset using pandas
# Reading the CSV file after converting file to csv and removing superfluous spaces via Excel.
df = pd.read_csv('https://raw.githubusercontent.com/ShelbyP27/DS7331-Project/main/adult-data.csv')

In [11]:
#Cleaning up data set
df = df.replace(to_replace='?',value=np.nan) # replace '?' with NaN (not a number)
df.dropna(inplace=True) # Removing na values
df.duplicated(subset=None, keep='first') #Remove duplicates
df['income'] = df['income'].map({'<=50K': 0, '>50K': 1}).astype(int) #One-hot respone

In [12]:
# One-hot encoding the Categorical variables
if 'sex' in df:
    df['IsMale'] = df.sex == 'Male'
    df.IsMale = df.IsMale.astype(np.int64)
    del df['sex']
    
if 'marital-status' in df:
    tmp_df = pd.get_dummies(df['marital-status'], prefix = 'Marital')
    df = pd.concat((df, tmp_df), axis =1)
    del df['marital-status']
    
if'relationship' in df:
    tmp_df = pd.get_dummies(df['relationship'], prefix = 'Rel')
    df = pd.concat((df, tmp_df), axis =1)
    del df['relationship']

if 'race' in df:
    tmp_df = pd.get_dummies(df['race'], prefix = 'Race')
    df = pd.concat((df, tmp_df), axis =1)
    del df['race']

if 'workclass' in df:
    tmp_df = pd.get_dummies(df['workclass'], prefix = 'Work')
    df = pd.concat((df, tmp_df), axis =1)
    del df['workclass']

if 'occupation' in df:
    tmp_df = pd.get_dummies(df['occupation'], prefix = 'Occupation')
    df = pd.concat((df, tmp_df), axis =1)
    del df['occupation']

if 'education' in df:
    tmp_df = pd.get_dummies(df['education'], prefix = 'Education')
    df = pd.concat((df, tmp_df), axis =1)
    del df['education']

    
#Replace Native Country with Immigrant atribute
if 'native-country' in df:
    df['immigrant'] = np.where(df['native-country']!= 'United-States', 1, 0)
    del df['native-country']

In [15]:
# Separating the features from the response
if 'income' in df:
    y = df['income'].values
    del df['income']
    X = df.values

# Train / Test split with scaled_X
scaled_X = StandardScaler().fit_transform(X)
x_train, x_test, y_train, y_test = train_test_split(scaled_X, y, test_size = .2, random_state=1)

In [16]:
Xnew = df[['age','hours-per-week', 'capital-loss','education-num','Marital_Never-married','Marital_Married-civ-spouse','Work_Private', 'Rel_Husband','capital-gain','Rel_Not-in-family']].values

newscaled_X = StandardScaler().fit_transform(Xnew)
x_train, x_test, y_train, y_test = train_test_split(newscaled_X, y, test_size = .2, random_state=1)

## Business Understanding
*Describe the purpose of the data set you selected (i.e., why was this data collected in the first place?). How will you measure the effectiveness of a good algorithm? Why does your chosen validation method make sense for this specific dataset and the stakeholders needs?*

The purpose of the dataset selected is to identify what characteristics an individual needs to possess in order to make an income exceeding $50,000. An effective algorithm will classify with good accuracy (.85 or above since previous models have had success at that level) and not be a burden computational (<5 minutes). A good algorithm should also be transparent to some degree, so we are able to easily identify what attributes contributed to the outcome of the prediction. 



## Data Understanding
*Part One: Describe the meaning and type of data (scale, values, etc.) for each attribute in the data file. Verify data quality: Are there missing values? Duplicate data? Outliers? Are those mistakes? How do you deal with these problems?*

The final dataset that will be used for classification of individuals with an income greater than 50k includes 63 predictor variables. Seven of the eight original categorical variables, including sex, marital status, relationship, race, workclass, occupation, and education, have been one-hot encoded, resulting in a larger number of predictor variables. The eighth original categorical variable, native country, has been trasnformed to a binary variable indicitive of immigrant status; records with native country as 'United States' were assigned a value of 0 while all else was assigned a value of 1. 

According to the Kohavi and Becker, the following changes have already been made to the raw data:
* Discretized agrossincome into two ranges with threshold 50,000.
* Convert U.S. to US to avoid periods.
* Convert Unknown to "?"
* Run MLC++ GenCVFiles to generate data,test.
<br>

Therefore, we began by looking for the '?' values in the dataset by counting the number in each column. Then, we replaced as null using numpy. We decided to handle missing values in the following three variables which possessed missing values as below: 

* Workclass: 1836 rows
* Occupation: 1843 rows
* Native-Country: 583 rows

There was not a way to impute values for these categorical variables and leaving as unknown variables does not add value to our analysis. Therefore, these rows were removed leaving us with 30,162 complete rows in this dataframe. As for duplicates, Kohavi and Becker described 6 duplicate rows. After reviewing our current dataframe, there do not appear to be any instances of duplicate data.

*Part Two: Visualize the any important attributes appropriately. Important: Provide an interpretation for any charts or graphs.*

----INSERT REVIEW OF ATTRIBUTE SELECTION PROCESS. MAY NEED TO REVISIT #1 & #2----

## Modeling and Evaluation
*Different tasks will require different evaluation methods. Be as thorough as possible when analyzing the data you have chosen and use visualizations of the results to explain the performance and expected outcomes whenever possible. Guide the reader through your analysis with plenty of discussion of the results.*
 

### Option A: Cluster Analysis

 - Perform cluster analysis using several clustering methods
 - How did you determine a suitable number of clusters for each method?
 - Use internal and/or external validation measures to describe and compare the clusterings and the clusters (some visual methods would be good).
 - Describe your results. What findings are the most interesting and why? 


In [ ]:
#kmeans
from sklearn.cluster import KMeans

X1 = newscaled_X

# run kmeans algorithm (this is the most traditional use of k-means)
kmeans = KMeans(init='random', # initialization
        n_clusters=100,  # number of clusters
        n_init=5)       # number of different times to run k-means
       
kmeans.fit(X1)

# visualize the data
centroids = kmeans.cluster_centers_
plt.plot(X1[:, 0], X1[:, 1], 'r.', markersize=2) #plot the data
plt.scatter(centroids[:, 0], centroids[:, 1],
            marker='+', s=200, linewidths=3, color='k')  # plot the centroids
plt.title('K-means clustering for X1')
plt.xlabel('X1, Feature 1')
plt.ylabel('X1, Feature 2')
plt.grid()
plt.show()

In [ ]:
from sklearn.cluster import MiniBatchKMeans

kmeans_mini = MiniBatchKMeans(n_clusters=2, batch_size=1000)
kmeans = KMeans(n_clusters=2)

print('Time for BatchKMeans:')
%time kmeans.fit(X1)
print('Time for MiniBatchKMeans:')
%time kmeans_mini.fit(X1)


# visualize the data
centroids = kmeans.cluster_centers_
plt.plot(X1[:, 0], X1[:, 1], 'r.', markersize=2) #plot the data
plt.scatter(centroids[:, 0], centroids[:, 1],
            marker='+', s=200, linewidths=3, label='Batch')  # plot the centroids

centroids = kmeans_mini.cluster_centers_
plt.scatter(centroids[:, 0], centroids[:, 1],
            marker='+', s=200, linewidths=3, color='k',label='Mini-Batch')  # plot the centroids
plt.legend()
plt.title('K-means clustering for X1')
plt.xlabel('X1, Feature 1')
plt.ylabel('X1, Feature 2')
plt.grid()
plt.show()

In [ ]:
from sklearn.cluster import DBSCAN

# now plot each dataset
plt.figure(figsize=(15,5))
for i,X in enumerate([X1]):
    plt.subplot(1,2,i+1)
    plt.plot(X[:, 0], X[:, 1], 'r.', markersize=2) #plot the data
    plt.title('Variable name: X{0}'.format(i+2))
    plt.xlabel('X{0}, Feature 1'.format(i+2))
    plt.ylabel('X{0}, Feature 2'.format(i+2))
    plt.grid()

plt.show()

In [ ]:
# lets first look at the connectivity of the graphs and distance to the nearest neighbors
from sklearn.neighbors import kneighbors_graph
X2 = X1

#=======================================================
# CHANGE THESE VALUES TO ADJUST MINPTS FOR EACH DATASET
X2_N = 5000

#=======================================================

# create connectivity graphs before calcualting the hierarchy
X2_knn_graph = kneighbors_graph(X2, X2_N, mode='distance') # calculate distance to four nearest neighbors 

N2 = X2_knn_graph.shape[0]
X2_4nn_distances = np.zeros((N2,1))
for i in range(N2):
    X2_4nn_distances[i] = X2_knn_graph[i,:].max()

X2_4nn_distances = np.sort(X2_4nn_distances, axis=0)


plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.plot(range(N2), X2_4nn_distances, 'r.', markersize=2) #plot the data
plt.title('Dataset name: X2, sorted by neighbor distance')
plt.xlabel('X2, Instance Number')
plt.ylabel('X2, Distance to {0}th nearest neighbor'.format(X2_N))
plt.grid()

plt.show()

### Option B: Association Rule Mining
 - Create frequent itemsets and association rules.
 - Use tables/visualization to discuss the found results.
 - Use several measure for evaluating how interesting different rules are.
 - Describe your results. What findings are the most compelling and why? 

In [21]:
# https://www.geeksforgeeks.org/python-sys-setrecursionlimit-method/ 
import sys


# Using sys.getrecursionlimit() method 
# to find the current recursion limit
limit = sys.getrecursionlimit()
  
# Print the current limit 
print('Before changing, limit of stack =', limit) 
  
# New limit
Newlimit = 500
  
# Using sys.setrecursionlimit() method 
sys.setrecursionlimit(Newlimit) 
  
# Using sys.getrecursionlimit() method 
# to find the current recursion limit
limit = sys.getrecursionlimit()
  
# Print the current limit 
print('After changing, limit of stack =', limit) 



Before changing, limit of stack = 3000
After changing, limit of stack = 500


In [22]:

%R -i X rules <- apriori(X,parameter = list(minlen=2, supp=0.05, conf=0.8))
%R rules.sorted <- sort(rules, by='lift')
%R plot(rules.sorted, method='grouped')

Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
        0.8    0.1    1 none FALSE            TRUE       5    0.05      2
 maxlen target  ext
     10  rules TRUE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 1508 

set item appearances ...[0 item(s)] done [0.00s].
set transactions ...[63 item(s), 30162 transaction(s)] done [0.05s].
sorting and recoding items ... [31 item(s)] done [0.01s].
creating transaction tree ... done [0.03s].
checking subsets of size 1 2 3 4 5 6 7 8 9 10 done [0.08s].
writing ... [15547 rule(s)] done [0.02s].
creating S4 object  ... done [0.02s].


RecursionError: maximum recursion depth exceeded while calling a Python object

In [ ]:
# prepare the data as a dataframe with boolean values
import pandas as pd

df = pd.DataFrame (
    [
        [True,True, True],
        [True, False,False],
        [True, True, True],
        [True, False, False],
        [True, True, True],
        [True, False, True],
        [True, True, True],
        [False, False, True],
        [False, True, True],
        [True, False, True],
    ],
    columns=list ('ABC')) 

# set up rpy2
from rpy2.robjects import pandas2ri
pandas2ri.activate()
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
arules = importr("arules")

# run apriori
itsets = arules.apriori(df, 
   parameter = ro.ListVector({"supp": 0.1, "target": "frequent itemsets"}))

# get itemsets as a dataframe
print(arules.DATAFRAME(itsets))

# get quality as a dataframe
print(itsets.slots["quality"])

# get itemsets as a matrix
itemset_as_matrix = ro.r('function(x) as(items(x), "matrix")')
itemset_as_matrix(itsets)

In [ ]:
#%R -o df_from_R df_from_R<-df

# now we have the exact same dataset as the one from R
# but it is now a pandas dataframe
#print(df_from_R.info())

### Option C: Collaborative Filtering
 - Create user-item matrices or item-item matrices using collaborative filtering
 - Determine performance of the recommendations using different performance measures and explain what each measure
 - Use tables/visualization to discuss the found results. Explain each visualization in detail.
 - Describe your results. What findings are the most compelling and why? 

## Deployment
 - Be critical of your performance and tell the reader how you current model might be usable by other parties. Did you achieve your goals? If not, can you reign in the utility of your modeling?
 - How useful is your model for interested parties (i.e., the companies or organizations that might want to use it)?
 - How would your deploy your model for interested parties?
 - What other data should be collected?
 - How often would the model need to be updated, etc.? 

## Exceptional Work
 - You have free reign to provide additional analyses or combine analyses 